In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("data.csv")
df.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


In [2]:
df = df.drop(['banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'], axis=1)

In [3]:
df = df.drop(['impressions'], axis=1)

In [4]:
df['date_time'] = df['date_time'].progress_apply(pd.to_datetime)

100%|██████████| 15821472/15821472 [21:06<00:00, 12496.58it/s]


In [5]:
df['day'] = df['date_time'].progress_apply(lambda x: x.day)
df['hour'] = df['date_time'].progress_apply(lambda x: x.hour)

100%|██████████| 15821472/15821472 [00:57<00:00, 275200.70it/s] 


In [6]:
df['weekday'] = df['date_time'].progress_apply(lambda x: x.weekday())
df['is_holiday'] = df['weekday'].progress_apply(lambda x: 1 if x in [5, 6] else 0)

100%|██████████| 15821472/15821472 [00:15<00:00, 1002510.48it/s]


In [7]:
first_banner_appearence = df.sort_values(by='date_time').groupby('banner_id').first()['day'].reset_index()
first_banner_appearence['days_ago'] = (first_banner_appearence['day'].apply(lambda x: x if x > 25 else x+30) - 32)*(-1)
df = df.merge(first_banner_appearence[['banner_id', 'days_ago']], on='banner_id')

In [8]:
df = df.drop(['date_time'], axis=1)

In [9]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

df_, df_val = df[df['day'] != 2], df[df['day'] == 2]
df_train, df_test = df_[:int(df_.shape[0]*0.9)], df_[int(df_.shape[0]*0.9):]
X_train = df_train.drop(['clicks'], axis=1)
y_train = df_train['clicks'].values
X_test = df_test.drop(['clicks'], axis=1)
y_test = df_test['clicks'].values
X_val = df_val.drop(['clicks'], axis=1)
y_val = df_val['clicks'].values

In [ ]:
from collections import defaultdict
from sklearn.model_selection import GroupShuffleSplit
import math
target = "clicks"
NUMERICAL_FEATURES = ["campaign_clicks"]
# в общем то все категориальные фичи здесь полезны в качестве филда, даже хеш юзера 
# ведь таким образом можно ухватить его взаимодействие с баннерами, зонами сайта и тп
CATEGORICAL_FEATURES = [
    feature for feature in df.columns if feature not in ["campaign_clicks", 'clicks']
]

encoder = {
    "currentcode": len(
        NUMERICAL_FEATURES
    ),
    "catdict": {},  
    "catcodes": {},
}  


# https://github.com/wngaw/blog/blob/master/xlearn_example/src/utils.py

def convert_to_ffm(path, df, type, target, numerics, categories, encoder):
    # Flagging categorical and numerical fields
    #print('convert_to_ffm - START')
    for x in numerics:
        if(x not in encoder['catdict']):
            #print(f'UPDATING CATDICT: numeric field - {x}')
            encoder['catdict'][x] = 0
    for x in categories:
        if(x not in encoder['catdict']):
            #print(f'UPDATING CATDICT: categorical field - {x}')
            encoder['catdict'][x] = 1

    nrows = df.shape[0]
    with open(path + str(type) + "_ffm.txt", "w") as text_file:

        # Looping over rows to convert each row to libffm format
        for n, r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow[target]))  # Set Target Variable here

            # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(encoder['catdict'].keys()):
                if(encoder['catdict'][x] == 0):
                    # Not adding numerical values that are nan
                    if math.isnan(datarow[x]) is not True:
                        datastring = datastring + " "+str(i)+":" + str(i)+":" + str(datarow[x])
                else:

                    # For a new field appearing in a training example
                    if(x not in encoder['catcodes']):
                        #print(f'UPDATING CATCODES: categorical field - {x}')
                        encoder['catcodes'][x] = {}
                        encoder['currentcode'] += 1
                        #print(f'UPDATING CATCODES: categorical value for field {x} - {datarow[x]}')
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    # For already encoded fields
                    elif(datarow[x] not in encoder['catcodes'][x]):
                        encoder['currentcode'] += 1
                        #print(f'UPDATING CATCODES: categorical value for field {x} - {datarow[x]}')
                        encoder['catcodes'][x][datarow[x]] = encoder['currentcode']  # encoding the feature

                    code = encoder['catcodes'][x][datarow[x]]
                    datastring = datastring + " "+str(i)+":" + str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

    # print('Encoder Summary:')
    # print(json.dumps(encoder, indent=4))
    return encoder


encoder = convert_to_ffm(
    "./data/",
    df_train,
    "train",
    target,
    NUMERICAL_FEATURES,
    CATEGORICAL_FEATURES,
    encoder,
)

encoder = convert_to_ffm(
    "./data/",
    df_test,
    "test",
    target,
    NUMERICAL_FEATURES,
    CATEGORICAL_FEATURES,
    encoder,
)

encoder = convert_to_ffm(
    "./data/",
    df_val,
    "val",
    target,
    NUMERICAL_FEATURES,
    CATEGORICAL_FEATURES,
    encoder,
)

In [ ]:
import os
# почему то логи обучения выводятся только в консоль
# так как xlearn не выводит перебираемый параметр - пользуюсь этой функцией
def log_to_console(k, reg):
    os.write(1, f"********************************************\n".encode())
    os.write(1, f"Current hyperparameters: k={k}, reg_lambda={reg}\n".encode())
    os.write(1, f"********************************************\n".encode())


In [ ]:
import xlearn as xl
import numpy as np

# на самом деле тут было [5, 10, 20, 50] - но ноут сдался на 50 и кернел упал
# но разница в итоге не сильно заметна, слишком грубая оценка при cv xlearn'a - сложно оценить кто действительно лучше
for k in [5, 10, 20]:
    log_to_console(k, 1e-2)
    ffm_model = xl.create_ffm()
    ffm_model.setTrain("./data/train_ffm.txt")
    ffm_model.setValidate("./data/test_ffm.txt")
    param = {'task':'binary', 'lr':0.01, 'lambda':1e-2, 'metric':'auc', 'epoch':20, 'k':k}
    ffm_model.cv(param)

У модели с размерностью k=10 лучший log-loss и auc

In [9]:
import xlearn as xl
import numpy as np
ffm_model = xl.create_ffm()
ffm_model.setTrain("./data/train_ffm.txt")
ffm_model.setValidate("./data/test_ffm.txt")
param = {'task':'binary', 'lr':0.01, 'lambda':0.01, 'metric':'auc', 'epoch':20, 'k':10}
ffm_model.fit(param, './model.out')

In [12]:
y_pred_zero_proba = y_train[y_train == 0].shape[0] / y_train.shape[0]
y_pred_ones_proba = y_train[y_train == 1].shape[0] / y_train.shape[0]

print("Log-loss by predicting class distribution of train dataset:")

log_loss(y_val, [[y_pred_zero_proba, y_pred_ones_proba]]*len(y_val))

Log-loss by predicting class distribution of train dataset:


0.15403798833827734

In [14]:
from sklearn.metrics import log_loss
ffm_model.setTest("./data/val_ffm.txt") 
# преобразуем в вероятности положительного класса
ffm_model.setSigmoid()
ffm_model.predict("./model.out", "./test_prediction.txt")

res = np.loadtxt('./test_prediction.txt')
log_loss(y_val,  res)

0.14517008166636636